In [19]:
import os
import requests
## Descargar dataset
# Directorio
_data_root = './data/covertype'
# Path 
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')
# Descaragar data
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    #https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
    url = 'https://docs.google.com/uc?export= \
    download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116203 entries, 0 to 116202
Data columns (total 13 columns):
 #   Column                              Non-Null Count   Dtype 
---  ------                              --------------   ----- 
 0   Elevation                           116203 non-null  int64 
 1   Aspect                              116203 non-null  int64 
 2   Slope                               116203 non-null  int64 
 3   Horizontal_Distance_To_Hydrology    116203 non-null  int64 
 4   Vertical_Distance_To_Hydrology      116203 non-null  int64 
 5   Horizontal_Distance_To_Roadways     116203 non-null  int64 
 6   Hillshade_9am                       116203 non-null  int64 
 7   Hillshade_Noon                      116203 non-null  int64 
 8   Hillshade_3pm                       116203 non-null  int64 
 9   Horizontal_Distance_To_Fire_Points  116203 non-null  int64 
 10  Wilderness_Area                     116203 non-null  object
 11  Soil_Type                           116

In [3]:
import tensorflow as tf

2025-02-24 21:43:37.173846: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-24 21:43:37.187073: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-24 21:43:37.401918: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-24 21:43:38.183818: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-24 21:43:40.694764: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [6]:
#SELECCIÓN DE CARACTERISTICAS
from sklearn.feature_selection import SelectKBest, f_classif
# Separar características y la variable objetivo
X = data.drop(columns=["Cover_Type"])  # Eliminar la columna objetivo
y = data["Cover_Type"]  # Variable objetivo

#Seleccionar solo las características numéricas
X_numeric = X.select_dtypes(include=['number'])


In [7]:
# Aplicar SelectKBest para selección de características
k = 5  # Número de características a seleccionar
selector = SelectKBest(score_func=f_classif, k=k)
X_selected = selector.fit_transform(X_numeric, y)

selected_features = X_numeric.columns[selector.get_support()]

print("Características seleccionadas:")
print(selected_features)

Características seleccionadas:
Index(['Elevation', 'Slope', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Horizontal_Distance_To_Fire_Points'],
      dtype='object')


In [10]:
#Crear DataFrame con True/False para cada columna
feature_selection_result = pd.DataFrame({
    "Columns": X_numeric.columns,
    "Retain": X_numeric.columns.isin(selected_features)
})

print(feature_selection_result.to_string(index=False))  # Imprimir en formato tabla

                           Columns  Retain
                         Elevation    True
                            Aspect   False
                             Slope    True
  Horizontal_Distance_To_Hydrology   False
    Vertical_Distance_To_Hydrology   False
   Horizontal_Distance_To_Roadways    True
                     Hillshade_9am    True
                    Hillshade_Noon   False
                     Hillshade_3pm   False
Horizontal_Distance_To_Fire_Points    True


In [11]:
#DATA PIPELINE
## Configurar el contexto interactivo

import tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

In [14]:
#Ingesta de datos
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.orchestration.metadata import sqlite_metadata_connection_config

# 📌 Definir la ruta del directorio de metadatos
PIPELINE_ROOT = os.path.join(os.getcwd(), "tfx_pipeline")
METADATA_PATH = os.path.join(PIPELINE_ROOT, "metadata.db")

# 📌 Crear la configuración de la base de datos SQLite en el formato correcto
metadata_connection_config = sqlite_metadata_connection_config(METADATA_PATH)

# 📌 Iniciar el contexto interactivo con la configuración corregida
context = InteractiveContext(pipeline_root=PIPELINE_ROOT, metadata_connection_config=metadata_connection_config)

print(f"Pipeline configurado en: {PIPELINE_ROOT}")
print(f"Base de datos de metadatos en: {METADATA_PATH}")


Pipeline configurado en: /app/tfx_pipeline
Base de datos de metadatos en: /app/tfx_pipeline/metadata.db


In [22]:
from tfx.components import CsvExampleGen

# Definir la ruta de los datos 
DATA_DIR = os.path.abspath("./data/covertype")  # Ruta absoluta del directorio donde está el CSV
print(f"Usando DATA_DIR: {DATA_DIR}")

#Crear el componente ExampleGen
example_gen = CsvExampleGen(input_base=DATA_DIR)

#Ejecutar ExampleGen en el contexto interactivo
context.run(example_gen)

#Verificar la salida de los datos ingeridos
artifact_uri = example_gen.outputs['examples'].get()[0].uri
print(f"Datos ingeridos y convertidos a TFRecord en: {artifact_uri}")



Usando DATA_DIR: /app/data/covertype


Datos ingeridos y convertidos a TFRecord en: /app/tfx_pipeline/CsvExampleGen/examples/5


In [25]:
#Estadisticas
from tfx.components import StatisticsGen

#Crear el componente StatisticsGen con la salida de ExampleGen
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples']
)

#Ejecutar StatisticsGen en el contexto interactivo
context.run(statistics_gen)

#Obtener la URI de las estadísticas generadas
artifact_uri = statistics_gen.outputs['statistics'].get()[0].uri
print(f"Estadísticas generadas en: {artifact_uri}")


Estadísticas generadas en: /app/tfx_pipeline/StatisticsGen/statistics/6


In [27]:
print("Contenido de StatisticsGen:", os.listdir(artifact_uri))

Contenido de StatisticsGen: ['Split-eval', 'Split-train']


In [28]:
train_stats_dir = os.path.join(artifact_uri, "Split-train")
print("Contenido de Split-train:", os.listdir(train_stats_dir))


Contenido de Split-train: ['FeatureStats.pb']


In [31]:
from google.protobuf import text_format
from tensorflow_metadata.proto.v0 import statistics_pb2

#Ruta del archivo de estadísticas en Split-train
stats_path = os.path.join(artifact_uri, "Split-train", "FeatureStats.pb")

#Verificar si el archivo existe antes de cargarlo
if os.path.exists(stats_path):
    #Crear un objeto ProtoBuf para almacenar las estadísticas
    stats_proto = statistics_pb2.DatasetFeatureStatisticsList()

    #Leer el archivo como un ProtoBuf binario
    with open(stats_path, "rb") as f:
        stats_proto.ParseFromString(f.read())

    #Mostrar las estadísticas cargadas
    #print(stats_proto)
else:
    print(f"El archivo {stats_path} no existe. Verifica el directorio de estadísticas.")

import tensorflow_data_validation as tfdv
from tensorflow_metadata.proto.v0 import statistics_pb2

# Convertir el ProtoBuf cargado en el formato correcto para visualización
stats_dataset = statistics_pb2.DatasetFeatureStatisticsList()
stats_dataset.datasets.extend([stats_proto.datasets[0]])

# Visualizar las estadísticas con FACETS
tfdv.visualize_statistics(stats_dataset)

